# Ejercicio 2 - Parcial

Considerar los datasets de la distancia anual entre Urano y Saturno (en unidades astronómicas)
y la generación de energía nuclear por hora en Brasil (kWh), encontrados en este sitio: https:
//tinyurl.com/2k3h8bz4 (los datos están a mitad de página).

### Ayuda

Si se usan funciones de librería scipy.stats, prestar atención a vectorized=False, paired=True

## a)
Encontrar el estimador plug-in del coeficiente de correlación entre ambos conjuntos de datos.

### Desarrollo

In [ ]:
# Aquí va código de Python

## b)
Estimar su error estándar usando bootstrap.

### Desarrollo

In [1]:
# Aquí va código de Python

## c)
Encontrar un intervalo de confianza de nivel 0.95 usando los métodos normal, percentile
bootstrap, basic bootstrap y BCa.

### Desarrollo

In [ ]:
# Aquí va código de Python